In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   

In [ ]:
# 加载Bert模型和tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
ISCX = pd.read_csv('./ISCX-URL2016.csv')
# 准备数据
# 假设urls和labels分别为URL列表和对应的标签列表
urls = ISCX['URL'].values
labels = [0 if i=='no'  else 1 for i in ISCX['Target'].values]  # 0代表正常，1代表恶意

In [ ]:
# 划分训练集和验证集
train_urls, val_urls, train_labels, val_labels = train_test_split(urls, labels, test_size=0.2, random_state=42)

In [ ]:
train_urls = train_urls.tolist()
val_urls  = val_urls.tolist()
train_urls = [str(i) for i in train_urls]
val_urls = [str(i) for i in val_urls]

In [ ]:
# 对URL进行tokenize
train_encodings = tokenizer(train_urls, truncation=True, padding=True)
val_encodings = tokenizer(val_urls, truncation=True, padding=True)

In [ ]:
# 转换为PyTorch tensor
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']), torch.tensor(train_labels))
val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']), torch.tensor(val_encodings['attention_mask']), torch.tensor(val_labels))

In [ ]:
# 定义DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [ ]:
# 定义优化器和损失函数
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# 冻结Bert模型的参数
for param in model.base_model.parameters():
    param.requires_grad = False


In [ ]:
model.to(device)

In [ ]:
# 训练模型
train_losses = []
val_losses = []
from tqdm import tqdm
for epoch in tqdm(range(5)):
    model.train()
    for input_ids, attention_mask, labels in train_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    predictions = []
    true_labels = []
    with torch.no_grad():
        for input_ids, attention_mask, labels in val_loader:
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            val_loss += loss.item()
            predictions.extend(torch.argmax(outputs.logits, dim=1).cpu().tolist())
            true_labels.extend(labels.cpu().tolist())
    val_losses.append(val_loss / len(val_loader))


In [ ]:
# 可视化训练过程
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 评估模型效果
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy}')
